## Tutorial smolagent
https://huggingface.co/docs/smolagents/guided_tour?Pick+a+LLM=HF+Inference+API

NOTA: Recomendable ejecutarlo en un entorno virtual de python

In [ ]:
!pip install smolagents python-dotenv 
#transformers accelerate

In [ ]:
!python.exe -m pip install --upgrade pip

In [6]:
from dotenv import load_dotenv
import os

# Cargar las variables de entorno del archivo .env
load_dotenv()

# Acceder a la clave privada
hf_key = os.getenv("HF_TOKEN")
#!huggingface-cli login --token $hf_key

In [7]:
model_id = "meta-llama/Llama-3.3-70B-Instruct" 
# model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

 ### 1. Usando HF Inference API

In [ ]:
from smolagents import CodeAgent, HfApiModel

model = HfApiModel(model_id=model_id, token=hf_key) # You can choose to not pass any model_id to HfApiModel to use a default free model
# you can also specify a particular provider e.g. provider="together" or provider="sambanova"
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

El resultado a veces sale bien y otras no.

En el siguiente ejemplo, se define la función de fibonacci como una tool y entonces ya da el resultado correcto:

In [ ]:
from smolagents import tool

@tool
def fibonacci(n: int) -> int:
    """
    Calcula el enésimo número de Fibonacci.

    Args:
        n: Posición del número de fibonacci a calcular.
    """

    a, b = 0, 1
    for _ in range(n - 1):
        a, b = b, a + b
    return a

model = HfApiModel(model_id=model_id, token=hf_key)

# Configurar el agente con la tool de Fibonacci
agent = CodeAgent(tools=[fibonacci], model=model, add_base_tools=True)

# Ejecutar la consulta al agente
response = agent.run("What is the 118th Fibonacci number?")

print("Response:", response)

Otra forma de definir la tool pero usando una class

In [ ]:
from smolagents import CodeAgent, HfApiModel
from smolagents.tools import Tool
import os

# Definir la clase Fibonacci heredando de Tool
class FibonacciTool(Tool):
    name = "fibonacci"  # Agregamos el atributo name
    description = "Calculates the nth Fibonacci number."
    inputs = {
        "n": {
            "type": "integer",
            "description": "The position in the Fibonacci sequence to calculate"
        }
    }
    output_type = "integer"  # Añadimos el tipo de salida
     
    def forward(self, n: int) -> int:
        """Calcula el enésimo número de Fibonacci."""
        a, b = 0, 1
        for _ in range(n - 1):
            a, b = b, a + b
        return a

# Crear una instancia de la herramienta
fibonacci_tool = FibonacciTool()


# Usar HfApiModel con una clave de Hugging Face
# hf_key = os.getenv("HF_TOKEN")  # Asegúrate de definir esta variable en tu entorno
# model_id = "meta-llama/Llama-3.3-70B-Instruct"  # Puedes cambiarlo por otro modelo disponible

model = HfApiModel(model_id=model_id, token=hf_key)

# Configurar el agente con la tool de Fibonacci
agent = CodeAgent(tools=[fibonacci_tool], model=model, add_base_tools=True)

# Ejecutar la consulta al agente
response = agent.run("What is the 118th Fibonacci number?")

print("Response:", response)


### Inspecting an agent run

Here are a few useful attributes to inspect what happened after a run:

agent.logs
Stores the fine-grained logs of the agent. At every step of the agent’s run, everything gets stored in a dictionary that then is appended to agent.logs.

Running agent.write_memory_to_messages() writes the agent’s memory as list of chat messages for the Model to view. This method goes over each step of the log and only stores what it’s interested in as a message: for instance, it will save the system prompt and task in separate messages, then for each step it will store the LLM output as a message, and the tool call output as another message. Use this if you want a higher-level view of what has happened - but not every log will be transcripted by this method.

In [ ]:
agent.logs

### Utilidades

### 1. Buscar en internet conDuckDuckGo

In [ ]:
from smolagents import DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()
print(search_tool("Who's the current president of Russia?"))